# Testing

In [1]:
import pandas as pd
import numpy as np
import urllib.request as ureq
from bs4 import BeautifulSoup

In [2]:
path = 'https://www.platt.com/platt-electric-supply/LED-Traditional-Lamps-A-Type/Philips-Lighting/9-5A19-LED-827-22-DIM-120V/product.aspx?zpid=169504'
html=ureq.urlopen(path)

soup=BeautifulSoup(html,'html.parser')
type(soup)

bs4.BeautifulSoup

### Basic Information

In [7]:
title=soup.title
print(title)

<title>
	Philips Lighting - 9.5A19/LED/827-22 DIM 120V, Lamps - A-Type, LED - Traditional, Bulbs,, Ballasts - Platt Electric Supply
</title>


In [8]:
print(soup.price)

None


In [9]:
# find price box, strips away the tag info and leaves the price
price=soup.find("span",{"class":"ProductPriceOrderBox"}).get_text(strip=True)
print(price)

$6.83


In [10]:
# find description for product
description=soup.find("span",{"id":"lblDetailDes"}).get_text(strip=True)
print(description)

Dimmable LED Lamp with warm glow effect, 9.5W, 120V, Bulb: A-Type (A19), Base: Medium Screw (E26), Lumens: 800, CRI: 80, Color Temperature: 2700-2200K. Energy Star Rated. Rated Average Life: 25,000 Hrs. *Philips Part # 45332-4* 60W Equivalent. Suitable for use in enclosed fixtures


### Table Information

In [11]:
# find second specification table
spectable=soup.find("table",{"class":"seoSpecTable2"})
print(spectable)

<table cellpadding="0" cellspacing="0" class="seoSpecTable2"><tr><td class="detailsGridCol1">Color Temperature:</td><td>2700 (Warm White), 2700 (Warm White), 2700 (Warm White)</td></tr><tr class="detailsGridAltRow"><td class="detailsGridCol1">Lumens:</td><td>800</td></tr><tr><td class="detailsGridCol1">Shape:</td><td>A19 (Arbitrary)</td></tr><tr class="detailsGridAltRow"><td class="detailsGridCol1">Wattage:</td><td>9.500</td></tr><tr><td class="detailsGridCol1">Volts AC:</td><td>120</td></tr><tr class="detailsGridAltRow"><td class="detailsGridCol1">Type:</td><td>Lamp, LED, Dimmable</td></tr></table>


In [12]:
# loop through spec table and grab all attributes
specrows=spectable.find_all('tr')
print(specrows)


[<tr><td class="detailsGridCol1">Color Temperature:</td><td>2700 (Warm White), 2700 (Warm White), 2700 (Warm White)</td></tr>, <tr class="detailsGridAltRow"><td class="detailsGridCol1">Lumens:</td><td>800</td></tr>, <tr><td class="detailsGridCol1">Shape:</td><td>A19 (Arbitrary)</td></tr>, <tr class="detailsGridAltRow"><td class="detailsGridCol1">Wattage:</td><td>9.500</td></tr>, <tr><td class="detailsGridCol1">Volts AC:</td><td>120</td></tr>, <tr class="detailsGridAltRow"><td class="detailsGridCol1">Type:</td><td>Lamp, LED, Dimmable</td></tr>]


In [14]:
# split rows into columns.

for row in specrows:
    row_td=row.find_all('td')
    print(row_td)


[<td class="detailsGridCol1">Color Temperature:</td>, <td>2700 (Warm White), 2700 (Warm White), 2700 (Warm White)</td>]
[<td class="detailsGridCol1">Lumens:</td>, <td>800</td>]
[<td class="detailsGridCol1">Shape:</td>, <td>A19 (Arbitrary)</td>]
[<td class="detailsGridCol1">Wattage:</td>, <td>9.500</td>]
[<td class="detailsGridCol1">Volts AC:</td>, <td>120</td>]
[<td class="detailsGridCol1">Type:</td>, <td>Lamp, LED, Dimmable</td>]


In [13]:
# improved method to grab columns within each row
import re

list_rows=[]
for row in specrows:
    cells=row.find_all('td')
    str_cells=str(cells)
    clean=re.compile('<.*?>')
    clean2=(re.sub(clean,'',str_cells))
    list_rows.append(clean2)
print(clean2)
type(clean2)
print(list_rows)

[Type:, Lamp, LED, Dimmable]
['[Color Temperature:, 2700 (Warm White), 2700 (Warm White), 2700 (Warm White)]', '[Lumens:, 800]', '[Shape:, A19 (Arbitrary)]', '[Wattage:, 9.500]', '[Volts AC:, 120]', '[Type:, Lamp, LED, Dimmable]']


In [24]:
df12=pd.DataFrame()

In [23]:
d=df11.append(list_rows)
d

,0
0,"[Color Temperature:, 2700 (Warm White), 2700 (..."
1,"[Lumens:, 800]"
2,"[Shape:, A19 (Arbitrary)]"
3,"[Wattage:, 9.500]"
4,"[Volts AC:, 120]"
5,"[Type:, Lamp, LED, Dimmable]"


In [30]:
df12=df12.append(d)

In [31]:
df12

,0
0,"[Color Temperature:, 2700 (Warm White), 2700 (..."
1,"[Lumens:, 800]"
2,"[Shape:, A19 (Arbitrary)]"
3,"[Wattage:, 9.500]"
4,"[Volts AC:, 120]"
5,"[Type:, Lamp, LED, Dimmable]"


In [17]:
list_rows

['[Color Temperature:, 2700 (Warm White), 2700 (Warm White), 2700 (Warm White)]',
 '[Lumens:, 800]',
 '[Shape:, A19 (Arbitrary)]',
 '[Wattage:, 9.500]',
 '[Volts AC:, 120]',
 '[Type:, Lamp, LED, Dimmable]']

In [18]:
df00=pd.concat(pd.DataFrame(list_rows))
df00

TypeError: first argument must be an iterable of pandas objects, you passed an object of type "DataFrame"

### Info to Dataframe

In [11]:
# pass array to a dataframe
df=pd.DataFrame(list_rows)
df.head()

,0
0,"[Color Temperature:, 2700 (Warm White)]"
1,"[Lumens:, 800]"
2,"[Shape:, A19 (Arbitrary)]"
3,"[Wattage:, 9.500]"
4,"[Volts AC:, 120]"


In [12]:
# use split function to separate headers from values

new_df=df[0].str.split(',',n=1,expand=True)
new_df.columns=['Name','Value']
print(new_df.columns)

Index(['Name', 'Value'], dtype='object')


In [13]:
# Clean up data

new_df['Name']=new_df['Name'].str.replace('[','')
new_df['Name']=new_df['Name'].str.replace(':','')
new_df['Value']=new_df['Value'].str.replace(']','')
new_df

,Name,Value
0,Color Temperature,2700 (Warm White)
1,Lumens,800
2,Shape,A19 (Arbitrary)
3,Wattage,9.500
4,Volts AC,120
5,Type,"Lamp, LED, Dimmable"


In [14]:
# transpose dataframe because the headers are on the rows
new_df=new_df.transpose()
new_df

,0,1,2,3,4,5
Name,Color Temperature,Lumens,Shape,Wattage,Volts AC,Type
Value,2700 (Warm White),800,A19 (Arbitrary),9.500,120,"Lamp, LED, Dimmable"


In [16]:
# replace headers with first row and delete first row

df2=new_df.rename(columns=new_df.iloc[0])
df3=df2.drop(df2.index[0])
df3

,Color Temperature,Lumens,Shape,Wattage,Volts AC,Type
Value,2700 (Warm White),800,A19 (Arbitrary),9.500,120,"Lamp, LED, Dimmable"


### Second Table

Repeat this process for the other spec table, then combine them

In [17]:
# find second specification table
spectable2=soup.find("table",{"class":"seoSpecTable1"})
print(spectable2)

<table cellpadding="0" cellspacing="0" class="seoSpecTable1"><tr><td class="detailsGridCol1">Platt Cat:</td><td class="detailsGridCol2">PHI9A19LED82722DIM</td></tr><tr class="detailsGridAltRow"><td class="detailsGridCol1">Platt Item:</td><td class="detailsGridCol2">0169504</td></tr><tr><td class="detailsGridCol1">Cat:</td><td class="detailsGridCol2">9.5A19/LED/827-22 DIM 120V</td></tr><tr class="detailsGridAltRow"><td class="detailsGridCol1">UPC:</td><td class="detailsGridCol2">046677455828</td></tr><tr><td class="detailsGridCol1">Category: </td><td class="detailsGridCol2"><a href="/platt-electric-supply/LED-Traditional/Lamps-A-Type/search.aspx?SectionID=10&amp;GroupID=136&amp;CatID=7571&amp;SubCatID=7572" title="Lamps - A-Type">Lamps - A-Type</a></td></tr><tr class="detailsGridAltRow"><td class="detailsGridCol1">Base:</td><td>Medium Screw (E26)</td></tr><tr><td class="detailsGridCol1">Color Rendering Index (CRI):</td><td>80</td></tr></table>


In [19]:
# loop through spec table and grab all attributes
specrows2=spectable2.find_all('tr')
print(specrows2)

[<tr><td class="detailsGridCol1">Platt Cat:</td><td class="detailsGridCol2">PHI9A19LED82722DIM</td></tr>, <tr class="detailsGridAltRow"><td class="detailsGridCol1">Platt Item:</td><td class="detailsGridCol2">0169504</td></tr>, <tr><td class="detailsGridCol1">Cat:</td><td class="detailsGridCol2">9.5A19/LED/827-22 DIM 120V</td></tr>, <tr class="detailsGridAltRow"><td class="detailsGridCol1">UPC:</td><td class="detailsGridCol2">046677455828</td></tr>, <tr><td class="detailsGridCol1">Category: </td><td class="detailsGridCol2"><a href="/platt-electric-supply/LED-Traditional/Lamps-A-Type/search.aspx?SectionID=10&amp;GroupID=136&amp;CatID=7571&amp;SubCatID=7572" title="Lamps - A-Type">Lamps - A-Type</a></td></tr>, <tr class="detailsGridAltRow"><td class="detailsGridCol1">Base:</td><td>Medium Screw (E26)</td></tr>, <tr><td class="detailsGridCol1">Color Rendering Index (CRI):</td><td>80</td></tr>]


In [24]:
# split rows into columns.

for row in specrows2:
    row_td=row.find_all('td')
    print(row_td)

[<td class="detailsGridCol1">Platt Cat:</td>, <td class="detailsGridCol2">PHI9A19LED82722DIM</td>]
[<td class="detailsGridCol1">Platt Item:</td>, <td class="detailsGridCol2">0169504</td>]
[<td class="detailsGridCol1">Cat:</td>, <td class="detailsGridCol2">9.5A19/LED/827-22 DIM 120V</td>]
[<td class="detailsGridCol1">UPC:</td>, <td class="detailsGridCol2">046677455828</td>]
[<td class="detailsGridCol1">Category: </td>, <td class="detailsGridCol2"><a href="/platt-electric-supply/LED-Traditional/Lamps-A-Type/search.aspx?SectionID=10&amp;GroupID=136&amp;CatID=7571&amp;SubCatID=7572" title="Lamps - A-Type">Lamps - A-Type</a></td>]
[<td class="detailsGridCol1">Base:</td>, <td>Medium Screw (E26)</td>]
[<td class="detailsGridCol1">Color Rendering Index (CRI):</td>, <td>80</td>]


In [25]:
# improved method to grab columns within each row
import re

list_rows2=[]
for row in specrows2:
    cells=row.find_all('td')
    str_cells=str(cells)
    clean=re.compile('<.*?>')
    clean2=(re.sub(clean,'',str_cells))
    list_rows2.append(clean2)
print(clean2)
type(clean2)
print(list_rows2)

[Color Rendering Index (CRI):, 80]
['[Platt Cat:, PHI9A19LED82722DIM]', '[Platt Item:, 0169504]', '[Cat:, 9.5A19/LED/827-22 DIM 120V]', '[UPC:, 046677455828]', '[Category: , Lamps - A-Type]', '[Base:, Medium Screw (E26)]', '[Color Rendering Index (CRI):, 80]']


In [27]:
# pass array to a dataframe
df_tbl2=pd.DataFrame(list_rows2)
df_tbl2

,0
0,"[Platt Cat:, PHI9A19LED82722DIM]"
1,"[Platt Item:, 0169504]"
2,"[Cat:, 9.5A19/LED/827-22 DIM 120V]"
3,"[UPC:, 046677455828]"
4,"[Category: , Lamps - A-Type]"
5,"[Base:, Medium Screw (E26)]"
6,"[Color Rendering Index (CRI):, 80]"


In [30]:
# use split function to separate headers from values

df2_tbl2=df_tbl2[0].str.split(',',n=1,expand=True)
df2_tbl2.columns=['Name','Value']
df2_tbl2

,Name,Value
0,[Platt Cat:,PHI9A19LED82722DIM]
1,[Platt Item:,0169504]
2,[Cat:,9.5A19/LED/827-22 DIM 120V]
3,[UPC:,046677455828]
4,[Category:,Lamps - A-Type]
5,[Base:,Medium Screw (E26)]
6,[Color Rendering Index (CRI):,80]


In [31]:
# Clean up data

df2_tbl2['Name']=df2_tbl2['Name'].str.replace('[','')
df2_tbl2['Name']=df2_tbl2['Name'].str.replace(':','')
df2_tbl2['Value']=df2_tbl2['Value'].str.replace(']','')
df2_tbl2

,Name,Value
0,Platt Cat,PHI9A19LED82722DIM
1,Platt Item,0169504
2,Cat,9.5A19/LED/827-22 DIM 120V
3,UPC,046677455828
4,Category,Lamps - A-Type
5,Base,Medium Screw (E26)
6,Color Rendering Index (CRI),80


In [32]:
# transpose dataframe because the headers are on the rows
df3_tbl2=df2_tbl2.transpose()
df3_tbl2

,0,1,2,3,4,5,6
Name,Platt Cat,Platt Item,Cat,UPC,Category,Base,Color Rendering Index (CRI)
Value,PHI9A19LED82722DIM,0169504,9.5A19/LED/827-22 DIM 120V,046677455828,Lamps - A-Type,Medium Screw (E26),80


In [34]:
# replace headers with first row and delete first row

df3_tbl2=df3_tbl2.rename(columns=df3_tbl2.iloc[0])
df4_tbl2=df3_tbl2.drop(df3_tbl2.index[0])
df4_tbl2

,Platt Cat,Platt Item,Cat,UPC,Category,Base,Color Rendering Index (CRI)
Value,PHI9A19LED82722DIM,0169504,9.5A19/LED/827-22 DIM 120V,046677455828,Lamps - A-Type,Medium Screw (E26),80


In [37]:
df4_tbl2.index

Index(['Value'], dtype='object')

### Combine Table Info

In [39]:
# merge this dataframe with the other

dfmer=df3.join(df4_tbl2)
dfmer

,Color Temperature,Lumens,Shape,Wattage,Volts AC,Type,Platt Cat,Platt Item,Cat,UPC,Category,Base,Color Rendering Index (CRI)
Value,2700 (Warm White),800,A19 (Arbitrary),9.500,120,"Lamp, LED, Dimmable",PHI9A19LED82722DIM,0169504,9.5A19/LED/827-22 DIM 120V,046677455828,Lamps - A-Type,Medium Screw (E26),80


In [16]:
title=soup.title

# find price box, strips away the tag info and leaves the price
price=soup.find("span",{"class":"ProductPriceOrderBox"}).get_text(strip=True)

# find description for product
description=soup.find("span",{"id":"lblDetailDes"}).get_text(strip=True)

prodinfo=[]
prodinfo.append(title)
prodinfo.append(description)
prodinfo.append(price)
df0=pd.DataFrame(prodinfo)
df0=df0.transpose()
df0.columns=['title','description','price']
df0

,title,description,price
0,<title> Philips Lighting - 9.5A19/LED/827-22...,"Dimmable LED Lamp with warm glow effect, 9.5W,...",$6.83


Now per page these micro dataframes will have different columns, so will the concat properly? Hopefully.

# Page Information Pull

In [3]:
#path = 'https://www.platt.com/platt-electric-supply/LED-Traditional-Lamps-A-Type/Philips-Lighting/9-5A19-LED-827-22-DIM-120V/product.aspx?zpid=169504'
path = 'https://www.platt.com/platt-electric-supply/Fluorescent-Tube-T8-Standard/Philips-Lighting/F15T8-Cool-White-Plus/product.aspx?zpid=48910'

html=ureq.urlopen(path)

soup=BeautifulSoup(html,'html.parser')
type(soup)

bs4.BeautifulSoup

In [34]:
## Need to add bits for accounting for info potentially being in first or second table
## actually no i dont, i designed it to grab everything in the table...yay! Should test
# all in a block
import re

title=soup.title

# find price box, strips away the tag info and leaves the price
price=soup.find("span",{"class":"ProductPriceOrderBox"}).get_text(strip=True)

# find description for product
description=soup.find("span",{"id":"lblDetailDes"}).get_text(strip=True)

prodinfo=[]
prodinfo.append(title)
prodinfo.append(description)
prodinfo.append(price)
df0=pd.DataFrame(prodinfo)
df0=df0.transpose()
df0.columns=['title','description','price']
df0.index=['Value']


# find specification table
spectable=soup.find("table",{"class":"seoSpecTable1"})

# loop through spec table and grab all attributes
specrows=spectable.find_all('tr')
    
# improved method to grab columns within each row
list_rows=[]
for row in specrows:
    cells=row.find_all('td')
    str_cells=str(cells)
    clean=re.compile('<.*?>')
    clean2=(re.sub(clean,'',str_cells))
    list_rows.append(clean2)
type(clean2)

# pass array to a dataframe
df1=pd.DataFrame(list_rows)

# use split function to separate headers from values
df1_1=df1[0].str.split(',',n=1,expand=True)
df1_1.columns=['Name','Value']

# Clean up data
df1_1['Name']=df1_1['Name'].str.replace('[','')
df1_1['Name']=df1_1['Name'].str.replace(':','')
df1_1['Value']=df1_1['Value'].str.replace(']','')

# transpose dataframe because the headers are on the rows
df1_1=df1_1.transpose()

# replace headers with first row and delete first row
df1_2=df1_1.rename(columns=df1_1.iloc[0])
df1_3=df1_2.drop(df1_2.index[0])

# find second specification table
spectable2=soup.find("table",{"class":"seoSpecTable2"})

# loop through spec table and grab all attributes
specrows2=spectable2.find_all('tr')

# improved method to grab columns within each row
list_rows2=[]
for row in specrows2:
    cells=row.find_all('td')
    str_cells=str(cells)
    clean=re.compile('<.*?>')
    clean2=(re.sub(clean,'',str_cells))
    list_rows2.append(clean2)
type(clean2)
    
# pass array to a dataframe
df2=pd.DataFrame(list_rows2)

# use split function to separate headers from values
df2_1=df2[0].str.split(',',n=1,expand=True)
df2_1.columns=['Name','Value']

# Clean up data
df2_1['Name']=df2_1['Name'].str.replace('[','')
df2_1['Name']=df2_1['Name'].str.replace(':','')
df2_1['Value']=df2_1['Value'].str.replace(']','')

# transpose dataframe because the headers are on the rows
df2_1=df2_1.transpose()

# replace headers with first row and delete first row
df2_2=df2_1.rename(columns=df2_1.iloc[0])
df2_3=df2_2.drop(df2_2.index[0])

# merge this dataframe with the other
dfmer=df1_3.join(df2_3)
dfmer=df0.join(dfmer)
dfmer

,title,description,price,Platt Cat,Platt Item,Cat,UPC,Category,Base,Color Rendering Index (CRI),Color Temperature,Lumens,Shape,Wattage,Color,Length,Type
Value,<title> Philips Lighting - F15T8/Cool White ...,"Fluorescent Lamp, 15W, Bulb: T8 (Tubular), Bas...",$15.11,PHIF15T8CW18HLRTL,0048910,F15T8/Cool White Plus,046677392079,T8 - Standard,2-Pin (G13),59,4100 (Cool White),870,T8 (Tubular),15,Cool White,18 in,"T8, Lamp, Fluorescent"
